In [1]:
# resavamo UFLP sa genetskim algoritmom
import random

In [2]:
# jedinka
class Individual:
    def __init__(self, cost, fixedCost):
        # kodiranje za jedinku
        numResources = len(fixedCost)
        # ovde cemo kao u klasicnom genetskom algoritmu da genetski kod jedinke gledamo kao niz 1 i 0  tj true/false
        self.code = [random.random() < 0.25 for _ in range(numResources)] # otprilike 1/4 da bude true, ona prica od pre
        self.correctNonFeasible();  # za ispravljanje nedopustivog resenja ako se slucajno desi
        self.fitness = self.calcFitness(cost, fixedCost)
    
    # ovo je overload za operator <, ovo nam treba jer cemo negde ispod raditi sortiranje, moglo je samo u
    # sortiranju da se prosledi key= neka lambda za poredjenje, ali je hteo da nam pokaze da ovo moze u pajtonu
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    # mi ovde pisemo ove neke dodatne funkcije za proveru da li je stanje dopustivo i za ispravljanje nedopustivih
    # to je jedan nacin da se ovo radi (ali moze biti nezgodno npr ako negde zaboravimo da pozovemo te metode)
    # drugi nacin je da dopustimo nevalidna stanja, ali da im dodelimo najmanji fitness prilikom njegovog racunanja
    # time cemo da postignemo da ce evolucija sama da izbaci nedopustiva resenja
    def isFeasible(self):
        for c in self.code:
            if c:
                return True
        return False
    
    def invert(self):
        randomResource = random.randrange(len(self.code))
        self.code[randomResource] = not self.code[randomResource]
        if self.isFeasible():
            return randomResource
        else:
            return -1
    
    def correctNonFeasible(self):
        for c in self.code:
            if c:
                return
        randomResource = random.randrange(len(self.code))
        self.code[randomResource] = True
        
    def calcFitness(self, cost, fixedCost):
        # fitnes je bolji sto je cena manja
        numUsers = len(cost)
        numResources = len(fixedCost)
        usedResources = [False for _ in range(numResources)]
        totalCost = 0
        for i in range(numUsers):
            minCost = float('inf') 
            resourceUsed = -1
            for j in range(numResources):
                if cost[i][j] < minCost and self.code[j]:  # code[j] je tacno kada je resurs uspostavljen
                    minCost = cost[i][j]
                    resourceUsed = j
            totalCost += minCost
            usedResources[resourceUsed] = True
        # u usedResources su sada resusri koji su uspostavljeni i imaju poentu
        # ako se desi ona situacija da je neki resurs beskoristan jer su svim ljudima blizi neki drugi,
        # ova linija ce imati smisla
        self.code = usedResources
        # treba da razmatramo i samu cenu uspostavljanja
        for (j, resource) in enumerate(self.code):
            if resource:
                totalCost += fixedCost[j]
        # u klasicnom genetskom, trazi se jedinka sa najvecim fitnesom. Nama fitnes ima veze sa cenom, ali 
        # nama je bolja jedinka koja ima manju cenu, tako da mozemo da transformisemo ovaj totalCost nekako 
        # da se poklapa sa znacenjem fitnesa (a i ne moramo). ovde to radimo za primer
        fitness = 1 / totalCost  # nikad nece biti deljenje nulom jer se uvek neki resurs dodeli
        return fitness

In [3]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for i in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]
        return cost, fixedCost

In [4]:
cost, fixedCost = readInput('uflp1.txt')

In [5]:
# koristicemo turnirsku selekciju
def selection(population):
    TOURNAMENT_SIZE = 5
    bestFitness = float('-inf')
    index = -1
    for i in range(TOURNAMENT_SIZE):
        randomIndividual = random.randrange(len(population))
        if population[randomIndividual].fitness > bestFitness:
            bestFitness = population[randomIndividual].fitness
            index = randomIndividual
    # ovo sto smo napisali moze  i u jednoj liniji i da se jos osiguramo da ne budu isti izabrani vise puta
    # ali svejedno
    # return max(random.sample(population, TOURNAMENT_SIZE))
    return index

In [6]:
# radimo jednopoziciono ukrstanje
def crossover(parent1, parent2, child1, child2):
    breakpoint = random.randrange(len(parent1.code))
    
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child2.code[:breakpoint] = parent2.code[:breakpoint]
    
    child1.code[breakpoint:] = parent2.code[breakpoint:]
    child2.code[breakpoint:] = parent1.code[breakpoint:]
    
    child1.correctNonFeasible()
    child2.correctNonFeasible()

In [7]:
# recimo da hocemo da postoji neki % mogucnost da se svaki bit promeni (i da je dozvoljeno da se bukvalno svi promene)
def mutation(individual):
    MUTATION_PROB = 0.03
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            individual.code[i] = not individual.code[i]
    individual.correctNonFeasible()

In [37]:
POPULATION_SIZE = 100
NUM_GENERATIONS = 10
# redimo da jednu petinu populacije  (sa najboljim fitnesom) samo prenosimo u narendu generaciju,
#a ostale radimo normalno ukrstanjem i mutacijom
ELITISM_SIZE = POPULATION_SIZE // 5

population = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]
# odlucili smo da na pocetku ova 'nova populacija' ne bude prazna pa da se na nju appenduje
# nego da vec postoje neke vrednosti pa se menja, jer je kao tako laske
# npr u crossoveru cemo da iskoristimo ovo da nebi pravili nove objekte nego vec izmenili postojece
newPopulation = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]

for i in range(NUM_GENERATIONS):
    # nas overload za < sortira rastuce, a mi hocemo da na pocetku populacije budu oni sa najboljim fitnesom
    population.sort(reverse=True)
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
    for j in range(ELITISM_SIZE, POPULATION_SIZE, 2):
        parent1Index = selection(population)   # nije strasno ako se desi da isti bude izabran 2 puta, male su sanse za to
        parent2Index = selection(population)
        
        crossover(population[parent1Index], population[parent2Index],    # parent1, parent2
                  newPopulation[j], newPopulation[j+1])                  # child1, child2
        
        mutation(newPopulation[j])     # cild1
        mutation(newPopulation[j+1])   # child2
        
        newPopulation[j].fitness = newPopulation[j].calcFitness(cost, fixedCost)
        newPopulation[j+1].fitness = newPopulation[j+1].calcFitness(cost, fixedCost)
        
    population = newPopulation
    
bestIndividual = max(population)
print(f'solution: {bestIndividual.code}, fitness: {bestIndividual.fitness}, cost: {1 / bestIndividual.fitness}')

solution: [True, False, False], fitness: 0.029411764705882353, cost: 34.0


In [9]:
# sada cemo za primer da probamo da kombinujemo genetski alg i simulirano kaljenje
# bice sve isto samo sto hocemo da na najbolju jedinku svake generacije primenimo simulirano kaljenje

In [10]:
def simulatedAnnealing(individual, cost, fixedCost, iters):
    for i in range(1, iters + 1):    # +1 da nebi imali deljenje nulom u else grani
        # promenimo malo individual (ovde cemo da invertujemo jedan random bit)
        resourceInverted = individual.invert()
        if resourceInverted < 0:
            continue
        newFitness = individual.calcFitness(cost, fixedCost)
        if newFitness > individual.fitness:
            individual.fitness = newFitness
        else:  
            # sa malom verovatnocom prihvatamo novo resenje iako je losije
            p = 1.0 / i ** 0.5
            q = random.uniform(0, 1)
            if p > q:
                # zadrzimo to novo koje je losije
                individual.fitness = newFitness
            else:
                # vratimo se na prethodno ovde
                individual.code[resourceInverted] = not individual.code[resourceInverted]

In [65]:
POPULATION_SIZE = 100
NUM_GENERATIONS = 10
ELITISM_SIZE = POPULATION_SIZE // 5

population = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]
newPopulation = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]

for i in range(NUM_GENERATIONS):
    population.sort(reverse=True)
#     for p in population:
#         print(p.code)
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
    for j in range(ELITISM_SIZE, POPULATION_SIZE, 2):
        parent1Index = selection(population)   
        parent2Index = selection(population)
        
        crossover(population[parent1Index], population[parent2Index],   
                  newPopulation[j], newPopulation[j+1])                
        
        mutation(newPopulation[j])   
        mutation(newPopulation[j+1])  
        
        newPopulation[j].fitness = newPopulation[j].calcFitness(cost, fixedCost)
        newPopulation[j+1].fitness = newPopulation[j+1].calcFitness(cost, fixedCost)
        
    # pusticemo samo par iteracija simuliranog kaljenja da probamo jos malo da popravimo jedinku
    # ova jedinka koju smo izabrali ne mora nuzno da bude najbolja, moglo je da se desi da smo u trenutnoj populaciji
    # dobili bolju jedinku ali posto u ovom trenutku nije jos uradjeno sortiranje te nove populacije, ovde na pcoetku niza
    # ce da stoji najbolja jedinka iz prosle iteracije
    # (naravno mogli smo i bukvalno da nadjemo najbolju pa na nju da primenjujemo, ali ovo je dovoljno za priemr)
    simulatedAnnealing(newPopulation[0], cost, fixedCost, iters=10)
        
    population = newPopulation
#     for p in population:
#         print(p.code)
    
bestIndividual = max(population)
print(f'solution: {bestIndividual.code}, fitness: {bestIndividual.fitness}, cost: {1 / bestIndividual.fitness}') 
# NOTE: negde ovde je zaboravljeno ispravljanje nevalidnog stanja jer moze da se desi da bude [False, False, False]
# proveri u zvanicnim materijalima da li je ispravljeno

solution: [False, False, False], fitness: 0.029411764705882353, cost: 34.0
